Consider a triangular tile with two nodes positioned on each side.
Each node is connected to another node by a single path.
Each tile therefore has three paths.
How many ways are there of connecting these nodes?
Consider that the tiles can be rotated.
A rotated tile does *not* represent a new tile.

If we arrange the triangle with the base horizontal, then examine the top left node.
There are 5 other points to which this node could be connected.
Once that choice is made, consider the next free node clockwise around the triangle.
This has 3 remaining points to which it could be connected.
This leaves just two points to be connected, so the last path has no choices to be made.
This suggests there are $5\times3 = 15$ possible tiles.
However, there are several duplicates, and inspection reveals only 7 unique tiles.

For the representation of a tile, let us consider the nodes around the tile in clockwise order.
The path from the "first" node can be represented as the number of steps clockwise around the perimeter to reach the node at the other end.
This value can be recorded as a number from 1 to 5.
Each node around the triangle can be recorded similarly.
This does mean each path is recorded twice (once from each end), so there is some redundancy in this representation.

Let's create some code to draw these tiles.

In [1]:
from ipycanvas import Canvas, Path2D
from math import sin
from math import cos
from math import pi
from math import ceil
from random import randint

def get_polygon_vertices(N, x0, y0, radius):
    vertices = []
    ext_angle = 2 * pi / N
    
    # calculate the vertices of the polygon
    # to get the bottom flat we start pointing downwards, and rotate half an angle
    angle = pi / 2 + ext_angle / 2
    
    for i in range(N):
        x = cos(angle) * radius + x0
        y = sin(angle) * radius + y0
        vertices.append((x,y))
        angle += ext_angle
    return vertices


def find_nodes(vertices, inner_vertices):
    nodes = []
    control_points = []
    # hard code for 2 nodes per side
    vp = vertices[-1]
    ivp = inner_vertices[-1]
    for v,iv in zip(vertices, inner_vertices):
        nodes.append(((2 * vp[0] + v[0])/3, (2 * vp[1] + v[1])/3))
        control_points.append(ivp)
        nodes.append(((2 * v[0] + vp[0])/3, (2 * v[1] + vp[1])/3))
        control_points.append(iv)
        vp = v
        ivp = iv
    return nodes, control_points
    

def draw_tiles(tiles):
    NUM_SIDES = len(tiles[0]) // 2
    UNIT_WIDTH = 150
    COLS = 7
    ROWS = ceil(len(tiles) / COLS)

    canvas = Canvas(width=UNIT_WIDTH*COLS, height=UNIT_WIDTH*ROWS)

    x = UNIT_WIDTH/2
    y = UNIT_WIDTH/2
    poly_radius = 0.4 * UNIT_WIDTH # poly will be 80% of max size

    for tile in tiles:
        vertices = get_polygon_vertices(NUM_SIDES, x, y, poly_radius)
        canvas.line_width = 2
        canvas.stroke_style = "black"
        canvas.fill_style = "aliceblue"
        canvas.fill_polygon(vertices)
        canvas.stroke_polygon(vertices)
        inner_poly = get_polygon_vertices(NUM_SIDES, x, y, poly_radius/3)
        nodes, control_points = find_nodes(vertices, inner_poly)
        canvas.stroke_style = "purple"
        canvas.line_width = 3

        # draw bezier curves between connected nodes
        for node, path_length in enumerate(tile):
            p1 = node
            p2 = p1 + path_length
            if not p2 < len(tile):
                # this path wraps around past node 5
                # ignore it since it is already drawn
                continue
            canvas.begin_path()
            canvas.move_to(*nodes[p1])
            canvas.bezier_curve_to(*control_points[p1], *control_points[p2], *nodes[p2])
            canvas.stroke()
        
        x += UNIT_WIDTH
        if x > canvas.width:
            # move to next row
            x = UNIT_WIDTH / 2
            y += UNIT_WIDTH
        
    return canvas

draw_tiles([(1,5,1,5,1,5)])


Canvas(height=150, width=1050)

Now consider that this representation can be *canonicalised* by rotating it so that the resulting 6-digit sequence has the lowest possible value.
The rotation can be effected by moving the last two digits to the beginning.
There are only two rotations possible before arriving back at the original orientation.

Let's look at three rotations of such a tile:

In [2]:
draw_tiles([(1,5,2,2,4,4), (2,2,4,4,1,5), (4,4,1,5,2,2)])

Canvas(height=150, width=1050)

Now let's generate every possible tile:

In [3]:
numSides = 3
numNodes = numSides * 2
# create a tile with no paths initially - all path lengths are set to zero
tile = numNodes * [0]
all_tiles = []

for i in range(1, numNodes):
    # i represents the clockwise step distance from the beginning of the first path to its end
    start, end = 0, i
    tile[0] = i
    tile[i] = numNodes - i
    for j in range (1, numNodes - 2):
        # use a list to keep track of which nodes have yet to be connected
        nodes = list(range(numNodes))
        nodes.remove(0)
        nodes.remove(i)
        # j represents the clockwise step distance from the beginning of second path to its end
        # but ignoring already-connected nodes
        start, end = nodes[0], nodes[j]
        # re-compute j taking into account already-connected nodes
        j = end - start
        tile[start] = j
        tile[end] = numNodes - j
        nodes.remove(start)
        nodes.remove(end);
        # compute the final path
        k = nodes[1] - nodes[0]
        tile[nodes[0]] = k
        tile[nodes[1]] = numNodes - k;
        # fix the path lengths in a tuple, and add it to the list
        all_tiles.append(tuple(tile))

draw_tiles(all_tiles)


Canvas(height=450, width=1050)

Now let's define how to canonicalize a tile, then eliminate duplicates.

In [4]:
def canonicalize_tile(tile):
    fixed = tuple(tile)
    min = fixed
    for _ in range(1, numSides):
        # rotate the tile
        tile = tile[-2:] + tile[0:numNodes - 2]
        fixed = tuple(tile)
        # swap min if this rotation is lower-valued
        if fixed < min: min = fixed
    return min


tileset = list(set([canonicalize_tile(tile) for tile in all_tiles]))
tileset.sort()


print(f"There are {len(tileset)} unique tiles.")
for tile in tileset:
    # TODO: draw the tiles!
    print("\t", tile)

draw_tiles(tileset)


There are 7 unique tiles.
	 (1, 5, 1, 5, 1, 5)
	 (1, 5, 2, 2, 4, 4)
	 (1, 5, 3, 1, 5, 3)
	 (2, 3, 4, 2, 3, 4)
	 (2, 4, 4, 1, 5, 2)
	 (3, 3, 3, 3, 3, 3)
	 (5, 1, 5, 1, 5, 1)


Canvas(height=150, width=1050)

Now what happens if the tiles are square? Or n-sided?

A recursive approach might help here, starting with the list of free nodes — e.g. for a square `[0,1,2,3,4,5,6,7]`.
Each call would remove two nodes from the list — the first and another one depending on the length of the path.
The shorter list would be passed into the recursive call.

When there are only two nodes left in the list, the recursion stops and the tile is complete.